In [1]:
!pip install google-api-python-client

In [2]:
!pip install google-api-python-client

In [3]:
pip install langdetect

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pytube

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade google-auth-oauthlib google-auth-httplib2

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install emoji==1.7

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install text2emotion

In [8]:
import pandas as pd
from googleapiclient.discovery import build
import numpy as np
from pytube import Playlist,extract
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import statistics
from langdetect import detect
import emoji
import text2emotion as te
import re
import datetime as datetime

[nltk_data] Downloading package stopwords to C:\Users\Ivan
[nltk_data]     Tai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Ivan
[nltk_data]     Tai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Ivan
[nltk_data]     Tai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
#https://www.youtube.com/watch?v=SwSbnmqk3zY

In [10]:
youtube=build("youtube","v3",developerKey=api_key)

In [ ]:
secrets_filename = 'secret_keys'
api_keys = {}
with open(secrets_filename, 'r') as f:
    api_keys = json.loads(f.read())

In [11]:
# Generate list of video_url from playlist
def video_id_list(url="https://www.youtube.com/watch?v=Jkj36B1YuDU&list=PL3-sRm8xAzY-w9GS19pLXMyFRTuJcuUjy",N=100):
    url_list=Playlist(url)
    url_list=url_list.video_urls
    video_id_list=[]
    for n in range(0,N):
        video_id=extract.video_id(url_list[n])
        video_id_list.append(video_id)
    video_id_list=np.array(video_id_list)
    return video_id_list 

In [12]:
#create object singer
singer=video_id_list()


In [13]:
# test the presence of video
def test_video():
    items_list=[]
    for video in singer:
        request = youtube.videos().list(part="statisitcs",id=video)
        response= request.execute()
        try:
           items=response["items"][0]["statistics"]["commentCount"]
           items=True 
        except:
            items=False
            break
        items_list.append(items)
        
    return list(enumerate(items_list))



In [14]:
# collect information of song
def sad_song_information(playlist=singer):
    df_information=pd.DataFrame(columns=["title","id","channel_title","date"])
    for video in playlist:
        request = youtube.videos().list(part="snippet",id=video)
        response= request.execute()
        items=response["items"][0]
        id=items["id"]
        snippet=items["snippet"]
        title=snippet["title"]
        channel_title=snippet['channelTitle']
        date=snippet["publishedAt"]
        dict={"title":title,"id":id,"channel_title":channel_title,"date":date}
        df=pd.DataFrame(dict,index=[0])
        df_information=pd.concat([df_information,df],ignore_index=True)
    return df_information

In [15]:
# collect information of rock_song
def sad_song_statistics(playlist=singer):
    df_statistics=pd.DataFrame(columns=["viewCount","likeCount","commentCount"])
    for video in playlist:
        request = youtube.videos().list(part="statistics",id=video)
        response= request.execute()
        statistics=response["items"][0]["statistics"]
        viewCount=statistics["viewCount"]
        try:
            likeCount=statistics["likeCount"]
        except:
            likeCount="NaN"
        try:
            commentCount=statistics["commentCount"]
        except:
            commentCount="NaN"
        dict={"viewCount":viewCount,"likeCount":likeCount,"commentCount":commentCount}
        df=pd.DataFrame(dict,index=[0])
        df_statistics=pd.concat([df_statistics,df],ignore_index=True)
    return df_statistics    
        

In [29]:
# merge the two dataframe
def merge():
    df_stat=sad_song_statistics()
    df_info=sad_song_information()
    for column in df_stat.columns:
        df_info[column]=df_stat[column]
    return df_info


In [30]:
df_sad_song=merge()

In [31]:
df_sad_song["title"].tolist()
#the general pattern of title is "song_name/musician"-"musician"

['Tom Odell - Another Love (Lyrics)',
 'Benson Boone - Beautiful Things (Official Music Video)',
 'James Arthur - Train Wreck (Lyrics)',
 'd4vd - Here With Me (Official Audio)',
 'Miley Cyrus - Used To Be Young (Official Video)',
 'Sam Smith - Fire On Fire (From "Watership Down")',
 'Billie Eilish - WILDFLOWER (Lyrics)',
 'Lady Gaga, Bruno Mars - Die With A Smile (Official Music Video)',
 'Billie Eilish - What Was I Made For? (Official Music Video)',
 "James Arthur - Car's Outside (Lyrics)",
 'Paloma Faith - Only Love Can Hurt Like This (Lyrics)',
 'Miley Cyrus - Angels Like You (Lyrics)',
 'Alan Walker, Emma Steinbakken - Not You (Lyrics)',
 'Lewis Capaldi - Wish You The Best (Official Video)',
 'Libianca - People (Official Video)',
 "Shawn Mendes - It'll Be Okay",
 'The Weeknd - DIE FOR YOU (Lyrics)',
 'Darin - Fade Away (Music Video)',
 'Jaymes Young - Happiest Year (Lyric Video)',
 'Dean Lewis - How Do I Say Goodbye (Official Video)',
 "Cian Ducrot - I'll Be Waiting (Official Lyric

In [32]:
# create musician name from channel_title by covering all possible regex pattern
df_sad_song["musician"]=df_sad_song["title"].str.extract("(.*)-")
df_sad_song["musician2"]=df_sad_song["title"].str.extract("(.*)–")
df_sad_song["musician"]=df_sad_song["musician"].fillna(df_sad_song["musician2"])
df_sad_song=df_sad_song.drop("musician2",axis=1)

In [33]:
# extract the song_title in terms of pattern -song name or –song name
df_sad_song["song_title1"]=df_sad_song["title"].str.extract("-(.*)")
df_sad_song["song_title2"]=df_sad_song["title"].str.extract("–(.*)")

df_sad_song["song_title1"]=df_sad_song["song_title1"].fillna(df_sad_song["song_title2"])
   

df_sad_song["song_title1"].tolist()



[' Another Love (Lyrics)',
 ' Beautiful Things (Official Music Video)',
 ' Train Wreck (Lyrics)',
 ' Here With Me (Official Audio)',
 ' Used To Be Young (Official Video)',
 ' Fire On Fire (From "Watership Down")',
 ' WILDFLOWER (Lyrics)',
 ' Die With A Smile (Official Music Video)',
 ' What Was I Made For? (Official Music Video)',
 " Car's Outside (Lyrics)",
 ' Only Love Can Hurt Like This (Lyrics)',
 ' Angels Like You (Lyrics)',
 ' Not You (Lyrics)',
 ' Wish You The Best (Official Video)',
 ' People (Official Video)',
 " It'll Be Okay",
 ' DIE FOR YOU (Lyrics)',
 ' Fade Away (Music Video)',
 ' Happiest Year (Lyric Video)',
 ' How Do I Say Goodbye (Official Video)',
 " I'll Be Waiting (Official Lyric Video)",
 ' Easy On Me (Official Lyric Video)',
 ' Romantic Homicide (Lyrics)',
 ' Pretty Baby (Official Music Video)',
 ' ceilings (official video)',
 ' I Lost You (Lyric Video)',
 nan,
 ' Arcade (Lyrics) ft. FLETCHER',
 ' Nobody Gets Me (Lyric Video)',
 " Evergreen (You Didn't Deserve Me

In [34]:
# extract the song_title in terms of pattern song title "(" or song title"[" or song title
df_sad_song["song_title3"]=df_sad_song["song_title1"].str.extract("(.*)\[")
df_sad_song["song_title4"]=df_sad_song["song_title1"].str.extract("(.*)")
df_sad_song["song_title"]=df_sad_song["song_title1"].str.extract("(.*)\(")


song_title_list=["song_title3","song_title4"]
for title in song_title_list:
    df_sad_song["song_title"]=df_sad_song["song_title"].fillna(df_sad_song[title])

for n in range(1,4):
   df_sad_song=df_sad_song.drop(f"song_title{n}",axis=1)


df_sad_song["song_title"].tolist()


[' Another Love ',
 ' Beautiful Things ',
 ' Train Wreck ',
 ' Here With Me ',
 ' Used To Be Young ',
 ' Fire On Fire ',
 ' WILDFLOWER ',
 ' Die With A Smile ',
 ' What Was I Made For? ',
 " Car's Outside ",
 ' Only Love Can Hurt Like This ',
 ' Angels Like You ',
 ' Not You ',
 ' Wish You The Best ',
 ' People ',
 " It'll Be Okay",
 ' DIE FOR YOU ',
 ' Fade Away ',
 ' Happiest Year ',
 ' How Do I Say Goodbye ',
 " I'll Be Waiting ",
 ' Easy On Me ',
 ' Romantic Homicide ',
 ' Pretty Baby ',
 ' ceilings ',
 ' I Lost You ',
 nan,
 ' Arcade ',
 ' Nobody Gets Me ',
 ' Evergreen ',
 ' Those Eyes ',
 ' i wish you cheated ',
 ' THE LONELIEST ',
 'Smith - Fingers Crossed ',
 ' Hold My Hand ',
 ' My Fault ',
 ' golden hour ',
 ' Boat ',
 ' Love Me More ',
 ' you were there for me ',
 ' Perfect ',
 ' Is There Someone Else? ',
 ' Runaway ',
 ' As The World Caves In ',
 ' If You Ever Change Your Mind',
 ' Let Somebody Go ',
 ' The Joker And The Queen ',
 ' Reckless ',
 ' Enchanted ',
 ' Talking T

In [35]:
df_sad_song["song_title"][df_sad_song["song_title"].isnull()]

26    NaN
Name: song_title, dtype: object

In [36]:
def check_null_value(df=df_sad_song):
    missing_data=df_sad_song.isnull()
    for columns in missing_data.columns.values.tolist():
        print(missing_data[columns].value_counts())

In [37]:
check_null_value()

title
False    100
Name: count, dtype: int64
id
False    100
Name: count, dtype: int64
channel_title
False    100
Name: count, dtype: int64
date
False    100
Name: count, dtype: int64
viewCount
False    100
Name: count, dtype: int64
likeCount
False    100
Name: count, dtype: int64
commentCount
False    100
Name: count, dtype: int64
musician
False    99
True      1
Name: count, dtype: int64
song_title4
False    99
True      1
Name: count, dtype: int64
song_title
False    99
True      1
Name: count, dtype: int64


In [38]:
df_sad_song=df_sad_song.drop("song_title4",axis=1)

In [39]:
# no musician in the title
df_sad_song[df_sad_song["musician"].isnull()]

,title,id,channel_title,date,viewCount,likeCount,commentCount,musician,song_title
26,this is what losing someone feels like (offici...,LYLbf7KlQb8,JVKE,2023-03-31T23:32:49Z,6559717,146197,10111,NaN,NaN


In [50]:
df_sad_song[df_sad_song["song_title"].isnull()]

,title,id,channel_title,date,viewCount,likeCount,commentCount,musician,song_title
26,this is what losing someone feels like (offici...,LYLbf7KlQb8,JVKE,2023-03-31T23:32:49Z,6559717,146197,10111,NaN,NaN


In [53]:
df_sad_song["musician"]=df_sad_song["musician"].fillna("JVKE")
df_sad_song["song_title"]=df_sad_song["song_title"].fillna("this is what losing someone feels like")

In [54]:
df_sad_song[df_sad_song["song_title"].isnull()]

,title,id,channel_title,date,viewCount,likeCount,commentCount,musician,song_title


In [55]:
def standard_date(Time):
    Time=Time.replace("T"," ")
    Time=Time.replace("Z","")
    Time=datetime.datetime.strptime(Time,"%Y-%m-%d %H:%M:%S")
    return Time

In [56]:
df_sad_song["date"]=df_sad_song["date"].apply(standard_date)

In [57]:
df_sad_song['Year'] = df_sad_song['date'].dt.year    

In [58]:
# drop the null value
df_sad_song=df_sad_song.dropna(axis=0).reset_index(drop=True)

In [59]:
df_sad_song

,title,id,channel_title,date,viewCount,likeCount,commentCount,musician,song_title,Year
0,Tom Odell - Another Love (Lyrics),Jkj36B1YuDU,7clouds,2020-09-05 19:11:17,295250766,2984182,75483,Tom Odell,Another Love,2020
1,Benson Boone - Beautiful Things (Official Musi...,Oa_RSwwpPaA,Benson Boone,2024-01-18 23:00:13,451253770,4077315,89238,Benson Boone,Beautiful Things,2024
2,James Arthur - Train Wreck (Lyrics),CveANi17YfU,Lazy Fox,2020-09-08 15:40:52,265609174,2338276,58810,James Arthur,Train Wreck,2020
3,d4vd - Here With Me (Official Audio),B7Y4LHbpXv0,d4vd,2022-07-18 17:31:19,116828268,868085,10933,d4vd,Here With Me,2022
4,Miley Cyrus - Used To Be Young (Official Video),IZ3XMOdOdKM,MileyCyrusVEVO,2023-08-25 04:00:11,67690332,1281424,44587,Miley Cyrus,Used To Be Young,2023
...,...,...,...,...,...,...,...,...,...,...
95,Billie Eilish - everything i wanted (Official ...,qCTMq7xvdXU,BillieEilishVEVO,2019-11-14 00:00:12,204177568,2957984,123869,Billie Eilish,everything i wanted,2019
96,Imagine Dragons - Birds (Animated Video),vOXZkm9p_zY,ImagineDragonsVEVO,2019-07-24 04:00:04,309277581,2930219,54681,Imagine Dragons,Birds,2019
97,Miley Cyrus - Slide Away (Official Video),rrvFv6j3-sM,MileyCyrusVEVO,2019-09-06 16:00:12,60471242,967497,33659,Miley Cyrus,Slide Away,2019
98,"Lady Gaga, Bradley Cooper - Shallow (from A St...",bo_efYhYU2A,LadyGagaVEVO,2018-09-27 16:30:01,1727230193,8429618,207413,"Lady Gaga, Bradley Cooper",Shallow (from A Star Is Born),2018


In [60]:
#check the duplicates
def check_duplicates_values():
    for column in df_sad_song.columns:
        duplicates=df_sad_song[df_sad_song[column].duplicated()]
    return column,duplicates

In [61]:
check_duplicates_values()

('Year',
                                                 title           id  \
 2                 James Arthur - Train Wreck (Lyrics)  CveANi17YfU   
 6                 Billie Eilish - WILDFLOWER (Lyrics)  QkVI2n7-60c   
 7   Lady Gaga, Bruno Mars - Die With A Smile (Offi...  kPa7bsKwL-c   
 8   Billie Eilish - What Was I Made For? (Official...  cW8VLC9nnTo   
 9               James Arthur - Car's Outside (Lyrics)  xUT3ZcbVWmQ   
 ..                                                ...          ...   
 95  Billie Eilish - everything i wanted (Official ...  qCTMq7xvdXU   
 96           Imagine Dragons - Birds (Animated Video)  vOXZkm9p_zY   
 97          Miley Cyrus - Slide Away (Official Video)  rrvFv6j3-sM   
 98  Lady Gaga, Bradley Cooper - Shallow (from A St...  bo_efYhYU2A   
 99  Dermot Kennedy - Outnumbered (Official Music V...  A48hOToMuRE   
 
          channel_title                date   viewCount likeCount commentCount  \
 2             Lazy Fox 2020-09-08 15:40:52   265609174

In [62]:
df_sad_song.to_csv("sad_song_ytvideo_statistics.csv",index=False)
